In [4]:
# Import useful packages
import os
import time
import pandas as pd
import glob
import numpy as np

# For loading/querying postgres
from sqlalchemy import create_engine
import psycopg2
from psycopg2 import connect

In [5]:
# Create SQL engine to load data to postgres for central usage
from sqlalchemy import create_engine
dbname = '697_temp'
user = 'postgres'
password = 'poRter!5067'
host = 'databasesec.cvhiyxfodl3e.us-east-2.rds.amazonaws.com'

engine = create_engine(f'postgresql://{user}:{password}@{host}/{dbname}')

### Loading Open Positions to Postgres

In [6]:
# Collect all CSVs in Open_Positions_Data folder
txtfiles = []
for file in glob.glob("Open_Positions_Data/*.csv"):
    txtfiles.append(file)
    
txtfiles

['Open_Positions_Data\\MF_MLP.csv']

In [7]:
# create temp_df to test loading; pick model not already in table
temp_df = pd.read_csv(txtfiles[0],index_col=0)
temp_df.head()

,key,Date,Ticker,Quantity,Basis,Purchase Price,Current Value,Last,% Gain,model
0,Positions_2019-10-01,2019-10-01,WMT,1509,173512.781110,114.985276,175343.711914,116.198616,0.0,MF_MLP
1,Positions_2019-10-01,2019-10-01,MA,552,148535.234612,269.085570,165250.465191,299.366785,0.0,MF_MLP
2,Positions_2019-10-01,2019-10-01,CVX,1149,123411.632941,107.407862,128340.370129,111.697450,0.0,MF_MLP
3,Positions_2019-10-01,2019-10-01,BA,289,108117.194742,374.107940,95015.653199,328.773887,0.0,MF_MLP
4,Positions_2019-10-01,2019-10-01,COST,242,66448.675717,274.581305,68182.437990,281.745612,0.0,MF_MLP


In [8]:
temp_df.columns

Index(['key', 'Date', 'Ticker', 'Quantity', 'Basis', 'Purchase Price',
       'Current Value', 'Last', '% Gain', 'model'],
      dtype='object')

In [10]:
# Initial dataframe needs to create a new table
#temp_df.to_sql('open_positions', engine)

In [11]:
# Append data to table
temp_df.to_sql('open_positions', engine, if_exists='append')

In [13]:
# Test with a load
def import_open_positions():
    conn = connect(dbname = '697_temp', user = 'postgres', host = 'databasesec.cvhiyxfodl3e.us-east-2.rds.amazonaws.com', password = 'poRter!5067')
    cur = conn.cursor()
    query = "SELECT * FROM open_positions"
    data = pd.read_sql_query(query,conn)
    data = data.sort_values(['key', 'model'])
    return data
open_pos_df = import_open_positions().drop('index',axis=1)

open_pos_df[open_pos_df['model']=='MF_MLP']


,key,Date,Ticker,Quantity,Basis,Purchase Price,Current Value,Last,% Gain,model
12499,Positions_2019-10-01,2019-10-01,WMT,1509,173512.781110,114.985276,175343.711914,116.198616,0.00,MF_MLP
12549,Positions_2019-10-01,2019-10-01,MA,552,148535.234612,269.085570,165250.465191,299.366785,0.00,MF_MLP
12599,Positions_2019-10-01,2019-10-01,CVX,1149,123411.632941,107.407862,128340.370129,111.697450,0.00,MF_MLP
12649,Positions_2019-10-01,2019-10-01,BA,289,108117.194742,374.107940,95015.653199,328.773887,0.00,MF_MLP
12699,Positions_2019-10-01,2019-10-01,COST,242,66448.675717,274.581305,68182.437990,281.745612,0.00,MF_MLP
...,...,...,...,...,...,...,...,...,...,...
23576,Positions_2021-06-30,2021-04-01,JPM,837,126364.559783,150.973190,128800.270084,153.883238,1.93,MF_MLP
23577,Positions_2021-06-30,2021-04-01,BAC,2209,86097.828666,38.975930,90607.657331,41.017500,5.24,MF_MLP
23578,Positions_2021-06-30,2021-04-01,WFC,1077,42223.219040,39.204475,48397.687500,44.937500,14.62,MF_MLP
23579,Positions_2021-06-30,2021-04-01,MS,523,40605.793017,77.640140,47579.925200,90.975000,17.18,MF_MLP


### Loading track records to postgres

In [14]:
# Collect all CSVs in Open_Positions_Data folder
txtfiles = []
for file in glob.glob("track_record/*.csv"):
    txtfiles.append(file)
    
txtfiles

['track_record\\MF_MLP.csv']

In [15]:
# create temp_df to test loading
temp_df = pd.read_csv(txtfiles[0],index_col=0)
temp_df.head()

,Date,Value,Val_ex_cash,model
0,2019-10-01,1000000.000000,998076.538294,MF_MLP
1,2019-10-02,979836.084382,977912.622676,MF_MLP
2,2019-10-03,977172.086691,975248.624985,MF_MLP
3,2019-10-04,992500.555450,990577.093744,MF_MLP
4,2019-10-07,995455.717966,993532.256260,MF_MLP


In [16]:
# Initial dataframe needs to create a new table
#temp_df.to_sql('track_record', engine)

In [17]:
# Append data to table
temp_df.to_sql('track_record', engine, if_exists='append')

In [18]:
# Test with a load
def import_track_record():
    conn = connect(dbname = '697_temp', user = 'postgres', host = 'databasesec.cvhiyxfodl3e.us-east-2.rds.amazonaws.com', password = 'poRter!5067')
    cur = conn.cursor()
    query = "SELECT * FROM track_record"
    data = pd.read_sql_query(query,conn)
    data = data.sort_values(['Date', 'model'])
    return data
track_df = import_track_record().drop('index',axis=1)

track_df[track_df['model']=='MF_MLP']


,Date,Value,Val_ex_cash,model
936,2019-10-01,1.000000e+06,9.980765e+05,MF_MLP
937,2019-10-02,9.798361e+05,9.779126e+05,MF_MLP
938,2019-10-03,9.771721e+05,9.752486e+05,MF_MLP
939,2019-10-04,9.925006e+05,9.905771e+05,MF_MLP
940,2019-10-07,9.954557e+05,9.935323e+05,MF_MLP
...,...,...,...,...
1378,2021-06-24,1.584954e+06,1.584680e+06,MF_MLP
1379,2021-06-25,1.585322e+06,1.585047e+06,MF_MLP
1380,2021-06-28,1.593296e+06,1.593021e+06,MF_MLP
1381,2021-06-29,1.606645e+06,1.606370e+06,MF_MLP


### Loading cash records to postgres

In [19]:
# Collect all CSVs in Open_Positions_Data folder
txtfiles = []
for file in glob.glob("Cash_Positions_Data/*.csv"):
    txtfiles.append(file)
    
txtfiles

['Cash_Positions_Data\\MF_MLP.csv']

In [20]:
# create temp_df to test loading
temp_df = pd.read_csv(txtfiles[0],index_col=0)
temp_df.tail()

,key,cash,model
436,cash_2021-06-24,274.909754,MF_MLP
437,cash_2021-06-25,274.909754,MF_MLP
438,cash_2021-06-28,274.909754,MF_MLP
439,cash_2021-06-29,274.909754,MF_MLP
440,cash_2021-06-30,274.909754,MF_MLP


In [21]:
# Initial dataframe needs to create a new table
#temp_df.to_sql('cash_record', engine, if_exists='append')

In [22]:
# Test with a load
def import_cash_record():
    conn = connect(dbname = '697_temp', user = 'postgres', host = 'databasesec.cvhiyxfodl3e.us-east-2.rds.amazonaws.com', password = 'poRter!5067')
    cur = conn.cursor()
    query = "SELECT * FROM cash_record"
    data = pd.read_sql_query(query,conn)
    data = data.sort_values(['model','key'])
    return data
cash_df = import_cash_record().drop('index',axis=1)

cash_df.tail()


,key,cash,model
418,cash_2021-03-11,5.490783,RF Reg_target_7_rebal_7_2017-01-01
419,cash_2021-03-12,5.490783,RF Reg_target_7_rebal_7_2017-01-01
420,cash_2021-03-15,5.490783,RF Reg_target_7_rebal_7_2017-01-01
421,cash_2021-03-16,5.490783,RF Reg_target_7_rebal_7_2017-01-01
422,cash_2021-03-17,5.490783,RF Reg_target_7_rebal_7_2017-01-01


In [23]:
cash_df.count()

key      936
cash     936
model    936
dtype: int64

### Loading Value Positions to postgres

In [24]:
# Collect all CSVs in Open_Positions_Data folder
txtfiles = []
for file in glob.glob("Val_Positions_Data/*.csv"):
    txtfiles.append(file)
    
txtfiles

['Val_Positions_Data\\MF_MLP.csv']

In [25]:
# create temp_df to test loading
temp_df = pd.read_csv(txtfiles[0],index_col=0)
temp_df.tail()

,key,value,model
436,val_2021-06-24,1.584954e+06,MF_MLP
437,val_2021-06-25,1.585322e+06,MF_MLP
438,val_2021-06-28,1.593296e+06,MF_MLP
439,val_2021-06-29,1.606645e+06,MF_MLP
440,val_2021-06-30,1.612500e+06,MF_MLP


In [26]:
# Initial dataframe needs to create a new table
#temp_df.to_sql('val_positions', engine,if_exists='append')

In [27]:
# Test with a load
def import_val_positions():
    conn = connect(dbname = '697_temp', user = 'postgres', host = 'databasesec.cvhiyxfodl3e.us-east-2.rds.amazonaws.com', password = 'poRter!5067')
    cur = conn.cursor()
    query = "SELECT * FROM val_positions"
    data = pd.read_sql_query(query,conn)
    data = data.sort_values(['model','key'])
    return data
val_df = import_val_positions().drop('index',axis=1)

val_df.tail()

,key,value,model
417,val_2021-03-11,129090.678412,RF Reg_target_7_rebal_7_2017-01-01
418,val_2021-03-12,130598.742099,RF Reg_target_7_rebal_7_2017-01-01
419,val_2021-03-15,132633.335128,RF Reg_target_7_rebal_7_2017-01-01
420,val_2021-03-16,132407.606240,RF Reg_target_7_rebal_7_2017-01-01
421,val_2021-03-17,132356.878907,RF Reg_target_7_rebal_7_2017-01-01
